In [31]:
%load_ext autoreload
%autoreload 2
from parse.parse_reactions import ReactionParser
from parse.parse_species import parse_species_string, parse_all_species
from parse.parse_rate_constant import RateConstantParser
from parse.parse_input import separate_input_file, parse_args_kwargs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
sections = separate_input_file('input/sample_input.txt')
species_section = sections['species']
species, equivalent_species = parse_all_species(species_section)
print(species)
print(equivalent_species)

{'general': [], 'species': [], 'rate constants': [], 'reactions': []}
[Species(name='I', type=<SpeciesType.UNIT: 1>, FW=50.0, c0=0.005), Species(name='R', type=<SpeciesType.UNIT: 1>, FW=25.0, c0=0.0), Species(name='A', type=<SpeciesType.UNIT: 1>, FW=100.0, c0=1.5), Species(name='B', type=<SpeciesType.UNIT: 1>, FW=200.0, c0=1.5), Species(name='P[R.A]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), Species(name='P[R.B]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), Species(name='P[A.A]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), Species(name='P[A.B]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), Species(name='P[B.A]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), Species(name='P[B.B]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), Species(name='D', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0)]
[EquivalentSpecies(name='P[-.A]', type=<SpeciesType.POLY: 2>, eq=['P[A.A]', 'P[B.A]']), EquivalentSpecies(name='P[-.B]', type=<SpeciesType.POLY: 2>, eq=['P[A.B]', 'P[B.B]'])]


In [33]:
species_string = 'U I      c0=0.005 FW=50.0'
parse_species_string(species_string)

Species(name='I', type=<SpeciesType.UNIT: 1>, FW=50.0, c0=0.005)

In [34]:
constants_section = sections['rate constants']
rcp = RateConstantParser()
rcp.parse_all_constants(constants_section)

In [35]:
reaction_string = 'PR 23P[A.A] + B -kpAAB-> 5P[A.B]'
rp = ReactionParser(species, equivalent_species, rcp)
rp.parse_reaction_string(reaction_string)

('PR',
 [ReactionSpecies(species=Species(name='P[A.A]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), coeff=23),
  ReactionSpecies(species=Species(name='B', type=<SpeciesType.UNIT: 1>, FW=200.0, c0=1.5), coeff=1)],
 [ReactionSpecies(species=Species(name='P[A.B]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), coeff=5)],
 ['kpAAB'], {'k': 1000.0})

In [36]:
reactions_section = sections['reactions']
all_reactions = rp.parse_all_reactions(reactions_section)
print(all_reactions)

[Rate Constant:['kd'], {'k': 100000.0, 'f': 0.5},
Unit Reactant Species: [ReactionSpecies(species=Species(name='I', type=<SpeciesType.UNIT: 1>, FW=50.0, c0=0.005), coeff=1)],
Poly Reactant Species: [],
Unit Product Species: [ReactionSpecies(species=Species(name='R', type=<SpeciesType.UNIT: 1>, FW=25.0, c0=0.0), coeff=2)],
Poly Product Species: []

, Rate Constant:['kpAAA'], {'k': 1000.0},
Unit Reactant Species: [ReactionSpecies(species=Species(name='R', type=<SpeciesType.UNIT: 1>, FW=25.0, c0=0.0), coeff=1), ReactionSpecies(species=Species(name='A', type=<SpeciesType.UNIT: 1>, FW=100.0, c0=1.5), coeff=1)],
Poly Reactant Species: [],
Unit Product Species: [],
Poly Product Species: [ReactionSpecies(species=Species(name='P[R.A]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), coeff=1)]

, Rate Constant:['kpBBB'], {'k': 1000.0},
Unit Reactant Species: [ReactionSpecies(species=Species(name='R', type=<SpeciesType.UNIT: 1>, FW=25.0, c0=0.0), coeff=1), ReactionSpecies(species=Species(name='B', t

In [37]:
str = "ktdBB A=1e5 E=1e-3"
r = rcp.parse_constants_string(str)
r


['ktdBB'], {'A': 100000.0, 'E': 0.001}

In [38]:
c = rcp.parse_reaction_constant("PR P[R.A] + A -kpAAA-> P[A.A]")
print(c)

['kpAAA'], {'k': 1000.0}


In [39]:
from itertools import product

# Given string format
template_string = 'P[-.A] + P[-.B] -kpABC-> D'

# Define possible replacements for "P[-.A]" and "P[-.B]"
replacements_A = ['P[A.A]', 'P[B.A]']
replacements_B = ['P[A.B]', 'P[B.B]']

# Generate all permutations
permutations = list(product(replacements_A, replacements_B))

# Create a list of strings with permutations
for p in permutations:
    print(p)
result_strings = [template_string.replace('P[-.A]', p[0]).replace('P[-.B]', p[1]) for p in permutations]

# Print the result
for idx, result_string in enumerate(result_strings, start=1):
    print(f'{idx}: {result_string}')


('P[A.A]', 'P[A.B]')
('P[A.A]', 'P[B.B]')
('P[B.A]', 'P[A.B]')
('P[B.A]', 'P[B.B]')
1: P[A.A] + P[A.B] -kpABC-> D
2: P[A.A] + P[B.B] -kpABC-> D
3: P[B.A] + P[A.B] -kpABC-> D
4: P[B.A] + P[B.B] -kpABC-> D


In [40]:
rp.modify_reactions_section(reactions_section)

['ID I -kd-> 2R',
 'IN R + A -kpAAA-> P[R.A]',
 'IN R + B -kpBBB-> P[R.B]',
 'PR P[R.A] + A -kpAAA-> P[A.A]',
 'PR P[R.A] + B -kpAAB-> P[A.B]',
 'PR P[R.B] + A -kpBBA-> P[B.A]',
 'PR P[R.B] + B -kpBBB-> P[B.B]',
 'PR P[A.A] + A -kpAAA-> P[A.A]',
 'PR P[A.A] + B -kpAAB-> P[A.B]',
 'PR P[A.B] + A -kpABA-> P[B.A]',
 'PR P[A.B] + B -kpABB-> P[B.B]',
 'PR P[B.A] + A -kpBAA-> P[A.A]',
 'PR P[B.A] + B -kpBAB-> P[A.B]',
 'PR P[B.B] + A -kpBBA-> P[B.A]',
 'PR P[B.B] + B -kpBBB-> P[B.B]',
 'DP P[B.B] -kdBB-> P[A.B] + B',
 'DP P[B.B] -kdBB-> P[B.B] + B',
 'TC P[A.A] + P[A.A] -ktcAA-> D',
 'TC P[A.A] + P[B.A] -ktcAA-> D',
 'TC P[B.A] + P[A.A] -ktcAA-> D',
 'TC P[B.A] + P[B.A] -ktcAA-> D',
 'TC P[A.A] + P[A.B] -ktcAB-> D',
 'TC P[A.A] + P[B.B] -ktcAB-> D',
 'TC P[B.A] + P[A.B] -ktcAB-> D',
 'TC P[B.A] + P[B.B] -ktcAB-> D',
 'TC P[A.B] + P[A.B] -ktcBB-> D',
 'TC P[A.B] + P[B.B] -ktcBB-> D',
 'TC P[B.B] + P[A.B] -ktcBB-> D',
 'TC P[B.B] + P[B.B] -ktcBB-> D',
 'TD P[A.A] + P[A.A] -ktcAA-> 2D',
 'TD P[